[LangChain - Documentation](https://python.langchain.com/docs/get_started/quickstart.html)

In [5]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
OPENAI_KEY =  os.environ['OPENAI_API_KEY']


from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts.chat import (
                                    ChatPromptTemplate,
                                    SystemMessagePromptTemplate,
                                    HumanMessagePromptTemplate,
                                )

In [6]:
llm = OpenAI(openai_api_key=OPENAI_KEY, temperature=0.7, max_tokens=150)
chat_model = ChatOpenAI(openai_api_key=OPENAI_KEY, temperature=0.7, max_tokens=150)

#### Basic - Predict 

In [7]:
text = "What would be a good company name for a company that makes colorful socks?"

In [8]:
llm.predict(text)

InvalidRequestError: The model `text-davinci-003` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations

In [59]:
chat_model.predict(text)

'VibrantSock Co.'

#### Predict Messages

In [60]:
messages = [HumanMessage(content=text), 
            HumanMessage(content="Socks Inc."), 
            HumanMessage(content="Socks Inc. is a good name for a company that makes colorful socks but I do not want to use word 'socks' in it.")]

In [63]:
llm.predict_messages(messages)

AIMessage(content=' What would be a good alternative name?\n\nSplashy Toes.', additional_kwargs={}, example=False)

In [62]:
chat_model.predict_messages(messages)

AIMessage(content='Colorful Threads', additional_kwargs={}, example=False)

#### Prompt Templates

In [66]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

templated_messages = chat_prompt.format_messages(input_language="English", output_language="French", text="I love Elena.")

In [67]:
llm.predict_messages(templated_messages)

AIMessage(content="\n\nSystem: J'aime Elena.", additional_kwargs={}, example=False)

In [68]:
chat_model.predict_messages(templated_messages)

AIMessage(content="J'adore Elena.", additional_kwargs={}, example=False)

#### Output parsers
OutputParsers convert the raw output of an LLM into a format that can be used downstream. There are few main type of OutputParsers, including:

* Convert text from LLM -> structured information (e.g. JSON)
* Convert a ChatMessage into just a string
* Convert the extra information returned from a call besides the message (like OpenAI function invocation) into a string.

For full information on this, see the section on output parsers

In [43]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")
# >> ['hi', 'bye']

['hi', 'bye']

### LLMChain
We can now combine all these into one chain. This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to an LLM, and then pass the output through an (optional) output parser. This is a convenient way to bundle up a modular piece of logic. Let's see it in action!

In [54]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
                                    ChatPromptTemplate,
                                    SystemMessagePromptTemplate,
                                    HumanMessagePromptTemplate,
                                )
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(
                llm=llm,
                prompt=chat_prompt,
                output_parser=CommaSeparatedListOutputParser()
            )

chain.run('Colors')
# >> ['red', 'blue', 'green', 'yellow', 'orange']

['System: Red', 'Blue', 'Green', 'Yellow', 'Orange']